In [1]:
TFLITE_FILENAME = 'efficientdet-lite-colors_final.tflite'
LABELS_FILENAME = 'band-labels_colors.txt'

In [2]:
#using tiling to detect bands instead

import collections

import numpy as np
from PIL import Image
from PIL import ImageDraw

from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

import cv2
from color_recognition_api import color_histogram_feature_extraction
from color_recognition_api import knn_classifier
import os
import os.path
import sys
import string
import random

Object = collections.namedtuple('Object', ['label', 'score', 'bbox'])
 
  
def tiles_location_gen(img_size, tile_size, overlap):
  """Generates location of tiles after splitting the given image according the tile_size and overlap.

  Args:
    img_size (int, int): size of original image as width x height.
    tile_size (int, int): size of the returned tiles as width x height.
    overlap (int): The number of pixels to overlap the tiles.

  Yields:
    A list of points representing the coordinates of the tile in xmin, ymin,
    xmax, ymax.
  """

  tile_width, tile_height = tile_size
  img_width, img_height = img_size
  h_stride = tile_height - overlap
  w_stride = tile_width - overlap
  for h in range(0, img_height, h_stride):
    for w in range(0, img_width, w_stride):
      xmin = w
      ymin = h
      xmax = min(img_width, w + tile_width)
      ymax = min(img_height, h + tile_height)
      yield [xmin, ymin, xmax, ymax]


def non_max_suppression(objects, threshold):
  """Returns a list of indexes of objects passing the NMS.

  Args:
    objects: result candidates.
    threshold: the threshold of overlapping IoU to merge the boxes.

  Returns:
    A list of indexes containings the objects that pass the NMS.
  """
  if len(objects) == 1:
    return [0]

  boxes = np.array([o.bbox for o in objects])
  xmins = boxes[:, 0]
  ymins = boxes[:, 1]
  xmaxs = boxes[:, 2]
  ymaxs = boxes[:, 3]

  areas = (xmaxs - xmins) * (ymaxs - ymins)
  scores = [o.score for o in objects]
  idxs = np.argsort(scores)

  selected_idxs = []
  while idxs.size != 0:

    selected_idx = idxs[-1]
    selected_idxs.append(selected_idx)

    overlapped_xmins = np.maximum(xmins[selected_idx], xmins[idxs[:-1]])
    overlapped_ymins = np.maximum(ymins[selected_idx], ymins[idxs[:-1]])
    overlapped_xmaxs = np.minimum(xmaxs[selected_idx], xmaxs[idxs[:-1]])
    overlapped_ymaxs = np.minimum(ymaxs[selected_idx], ymaxs[idxs[:-1]])

    w = np.maximum(0, overlapped_xmaxs - overlapped_xmins)
    h = np.maximum(0, overlapped_ymaxs - overlapped_ymins)

    intersections = w * h
    unions = areas[idxs[:-1]] + areas[selected_idx] - intersections
    ious = intersections / unions

    idxs = np.delete(
        idxs, np.concatenate(([len(idxs) - 1], np.where(ious > threshold)[0])))


  return selected_idxs


def draw_object(draw, obj):
  """Draws detection candidate on the image.

  Args:
    draw: the PIL.ImageDraw object that draw on the image.
    obj: The detection candidate.
  """
  draw.rectangle(obj.bbox, outline='red')
  draw.text((obj.bbox[0], obj.bbox[3]), obj.label, fill='#0000')
  draw.text((obj.bbox[0], obj.bbox[3] + 10), str(obj.score), fill='#0000')


def reposition_bounding_box(bbox, tile_location):
  """Relocates bbox to the relative location to the original image.

  Args:
    bbox (int, int, int, int): bounding box relative to tile_location as xmin,
      ymin, xmax, ymax.
    tile_location (int, int, int, int): tile_location in the original image as
      xmin, ymin, xmax, ymax.

  Returns:
    A list of points representing the location of the bounding box relative to
    the original image as xmin, ymin, xmax, ymax.
  """
  bbox[0] = bbox[0] + tile_location[0]
  bbox[1] = bbox[1] + tile_location[1]
  bbox[2] = bbox[2] + tile_location[0]
  bbox[3] = bbox[3] + tile_location[1]
  return bbox

In [15]:
import tflite_runtime.interpreter as tflite 
import multiprocessing

interpreter = tflite.Interpreter(TFLITE_FILENAME,num_threads=multiprocessing.cpu_count())
interpreter.allocate_tensors()
labels = read_label_file(LABELS_FILENAME)

In [17]:

#using tiling to detect bands instead
    #ARGS HERE!!!

def sort_contours(cnts, method="top-to-bottom"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0

    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True

    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1

    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [c.bbox for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))

    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

img_path = 'Resistor_Lab_46.jpg'


tile_overlap = 15
#tile_sizes = "160x120"
score_threshold = .2
iou_threshold = .3
#ARGS ENDED HERE!!!


# Open image.
image = Image.open(img_path).convert('RGB')

draw = ImageDraw.Draw(image)

objects_by_label = dict()
img_size = image.size
x_tile = int(img_size[0]/2)
y_tile = int(img_size[1]/2)
tile_sizes = str(x_tile)+"x"+str(y_tile)
print(tile_sizes)
tile_sizes = [
    map(int, tile_size.split('x')) for tile_size in tile_sizes.split(',')
]

image2 = Image.open(img_path)

for tile_size in tile_sizes:

  for tile_location in tiles_location_gen(img_size, tile_size,
                                          tile_overlap):
    print(1)
    tile = image.crop(tile_location)
    _, scale = common.set_resized_input(
        interpreter, tile.size,
        lambda size, img=tile: img.resize(size, Image.NEAREST))
    interpreter.invoke()
    objs = detect.get_objects(interpreter, score_threshold, scale)
    for obj in objs:
      bbox = [obj.bbox.xmin, obj.bbox.ymin, obj.bbox.xmax, obj.bbox.ymax]
      bbox = reposition_bounding_box(bbox, tile_location)
      label = labels.get(obj.id, '')
      print(label)
      objects_by_label.setdefault(label,
                                  []).append(Object(label, obj.score, bbox))
bands = []
letters = string.ascii_lowercase
new_objs=[]

for label, objects in objects_by_label.items():
  idxs = non_max_suppression(objects, iou_threshold)
  for idx in idxs:
   # bbox = objects[idx].bbox
    new_objs.append(objects[idx])

objects=sort_contours(new_objs)
for label, objects in objects_by_label.items():
  #idxs = non_max_suppression(objects, iou_threshold)
  print(idxs)
  #for idx in idxs:
  for idx in range(len(objects)):
    draw_object(draw, objects[idx])

    bbox = objects[idx].bbox


    img_crop = image2.crop(box=bbox)
    strs = ''.join(random.choice(letters) for i in range(10))
    strs = strs+".jpg"
    #img_crop.save(strs)

    opencvImage = cv2.cvtColor(np.array(img_crop), cv2.COLOR_RGB2BGR)
      # checking whether the training data is ready
    
          

    # get the prediction
    color_histogram_feature_extraction.color_histogram_of_test_image(opencvImage)
    prediction = knn_classifier.main('training.data', 'test.data')
    print('Detected color is:', prediction)
    bands.append(prediction)



image

80x60
1


C:\Users\georg\AppData\Local\Temp\ipykernel_16604\4170727870.py:62: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  lambda size, img=tile: img.resize(size, Image.NEAREST))


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


KeyboardInterrupt: 

In [87]:
i=3
#print(bands[i])
gimg[i]


In [9]:

bands.reverse()
colors = ["black","brown","red","orange","yellow","green","blue","violet","gold"]
values = [0,1,2,3,4,5,6,7,-1]
print(bands)
def color2res(bands,colors,values):
    flag=0
    if bands == []:
        pass
    else:
        if(bands[0]=="Gold"):
          bands.reverse()
          flag=1  
        if(len(bands)==4):
            resistance =  (values[colors.index(bands[0])]*10 + values[colors.index(bands[1])]) * pow(10,(values[colors.index(bands[2])]))
        else:
            resistance =  (values[colors.index(bands[0])]*100 + values[colors.index(bands[1])]*10+values[colors.index(bands[2])]) * pow(10,(values[colors.index(bands[2])]))

        if flag==1:
          bands.reverse()
        return resistance
  
resistance = color2res(bands,colors,values)
resistance

['black', 'red', 'gold', 'brown']


0.2

In [66]:
import matplotlib.pyplot as plt

display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
#image = image.resize((display_width, int(display_width * height_ratio)))
#draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)

#draw = ImageDraw.Draw(image) 
  
  
text = str(resistance)
  
# drawing text size
draw.text((5, 5), text, align ="left") 

print(bands)
print(text)
image_original = Image.open(img_path)

f = plt.figure()
f.add_subplot(1,2, 1)
plt.imshow(np.rot90(image_original,2))
f.add_subplot(1,2, 2)
plt.imshow(np.rot90(image,2))
plt.show(block=True)




NameError: name 'resistance' is not defined

In [ ]:
#!pip install opencv-python-headless==4.1.2.30

In [65]:
import numpy as np
import cv2
open_cv_bands=[]
for i in bands:
  open_cv_image = np.array(i) 
  # Convert RGB to BGR 
  open_cv_bands.append(open_cv_image[:, :, ::-1].copy())

#ur script is below

#"Black","Brown","Grey","White"
colors = ["Black","Brown","Red","Orange","Yellow","Green","Blue","Violet","Gold"]
values = [0,1,2,3,4,5,6,7,-1]
all_lower = np.array([[0,22,13],[5,40,48],[0,100,100],[10,150,60],[30,90,45],[40,110,40],[94,80,2],[165,150,65],[13,125,95]])
all_higher = np.array([[50,255,40],[23,255,90],[10,255,255],[20,180,100],[50,255,255],[60,150,65],[126,255,255],[180,255,255],[18,150,150]])

final_colors=[]
for i in open_cv_bands:
  band_image = np.array(i)
  #result=image.copy()
  #cv2.imshow("Original", image)
    
  hsv = cv2.cvtColor(band_image, cv2.COLOR_BGR2HSV)
  
  max=0
  save_i=0
  for i in range(len(colors)):
      # lower boundary RED color range values; Hue (0 - 10)
      lower = all_lower[i]
      upper = all_higher[i]

      
      mask = cv2.inRange(hsv, lower, upper)
      if(i==0):
          save_mask=mask.copy()
      #result = cv2.bitwise_and(image, image, mask=mask)
      if(np.sum(mask)>max):
          max=np.sum(mask)
          save_i=i
          save_mask=mask.copy()




  final_colors.append(colors[save_i])

  #result = cv2.bitwise_and(result, result, mask=save_mask)
  # cv2.imshow('mask', save_mask)
  # cv2.imshow('result',result)

  # print(max)
  # cv2.waitKey(0)
  # cv2.destroyAllWindows()



IndexError: too many indices for array: array is 0-dimensional, but 3 were indexed